In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import math

np.random.seed(42)
random.seed(42)

# ------------------------------------------
# 보육자 등급별 설정
# ------------------------------------------
group_info = {
    1: {"members": [1,2,3], "k":0.002, "delta":(12,18)},
    2: {"members": [4,5,6,7], "k":0.003, "delta":(8,14)},
    3: {"members": [8,9,10], "k":0.004, "delta":(4,10)},
}

# ------------------------------------------
# 점수 생성
# ------------------------------------------
def bounded_int(a,b):
    return random.randint(a,b)

def generate_scores(group):
    if group == 1:
        tp = bounded_int(4,5)
        sm = bounded_int(4,5)
        pa = bounded_int(3,5)
        cc = bounded_int(3,5)
        cs = bounded_int(3,5)
    elif group == 2:
        cc = bounded_int(4,5)
        cs = bounded_int(4,5)
        other = lambda: bounded_int(2,4)
        tp, pa, sm = other(), other(), other()
    else:
        tp = bounded_int(1,3)
        sm = bounded_int(1,3)
        oc = lambda: bounded_int(2,4)
        pa, cc, cs = oc(), oc(), oc()

    return {
        "time_punctuality": tp,
        "preparedness_activity": pa,
        "communication_with_child": cc,
        "safety_management": sm,
        "communication_skill": cs,
    }

# ------------------------------------------
# 후기 생성
# ------------------------------------------
def review_text(avg):
    if avg >= 4:
        return "아이가 잘 따르고 소통도 좋았습니다."
    elif avg >= 3:
        return "전반적으로 괜찮았고 다음에 또 이용할 것 같아요."
    else:
        return "시간 약속이 아쉬웠고 개선이 필요합니다."

# ------------------------------------------
# 전체 날짜 생성: 2023.01.01 ~ 2024.12.31
# ------------------------------------------
start_date = datetime(2023,1,1)
end_date   = datetime(2024,12,31)
total_days = (end_date - start_date).days + 1

rows = []

# ------------------------------------------
# 각 보육자별 시계열 생성
# ------------------------------------------
for caregiver_id in range(1, 11):

    # 그룹 찾기
    group = [g for g,info in group_info.items() if caregiver_id in info["members"]][0]
    k = group_info[group]["k"]
    delta_range = group_info[group]["delta"]

    prev_prob = None
    days_since = 0
    match_count = 0

    # 2023~2024 전체 날짜 반복
    for i in range(total_days):
        current_date = start_date + timedelta(days=i)
        if prev_prob is None:
            rematched_today = 1
            prob = 0
            prev_prob = prob
            days_since = 0
            match_count = 1

        else:
            days_since += 1

            prob = prev_prob * math.exp(-k * 1)

            rematched_today = 1 if random.random() < 0.18 else 0

            if rematched_today == 1:
                match_count += 1

                if match_count == 2:
                    prob = 100
                else:
                    if prev_prob == 100:
                        prob = 100
                    else:
                        delta = random.randint(*delta_range)
                        prob = min(100, prob + delta)

                days_since = 0

            prev_prob = prob

        # 점수 생성
        scores = generate_scores(group)
        avg_score = np.mean(list(scores.values()))

        rows.append({
            "date": current_date.strftime("%Y-%m-%d"),
            "caregiver_id": caregiver_id,
            "caregiver_group": group,
            **scores,
            "review_text": review_text(avg_score),
            "days_since_last_rematch": days_since,
            "rematched_today": rematched_today,
            "rematch_probability": round(prob, 2)
        })

# ------------------------------------------
# CSV 저장
# ------------------------------------------
df = pd.DataFrame(rows)
df.to_csv("care_matching_timeseries_2023_2024.csv", index=False)

print("완료! → care_matching_timeseries_2023_2024.csv 생성됨")
df.head()

완료! → care_matching_timeseries_2023_2024.csv 생성됨


,date,caregiver_id,caregiver_group,time_punctuality,preparedness_activity,communication_with_child,safety_management,communication_skill,review_text,days_since_last_rematch,rematched_today,rematch_probability
0,2023-01-01,1,1,4,5,4,4,3,아이가 잘 따르고 소통도 좋았습니다.,0,1,0.0
1,2023-01-02,1,1,4,5,4,4,3,아이가 잘 따르고 소통도 좋았습니다.,1,0,0.0
2,2023-01-03,1,1,4,5,5,4,3,아이가 잘 따르고 소통도 좋았습니다.,0,1,100.0
3,2023-01-04,1,1,5,4,5,4,4,아이가 잘 따르고 소통도 좋았습니다.,1,0,99.8
4,2023-01-05,1,1,4,5,4,4,4,아이가 잘 따르고 소통도 좋았습니다.,2,0,99.6


In [ ]:
CSV_PATH = "care_matching_timeseries_2023_2024.csv"
df = pd.read_csv(CSV_PATH)

print("🔍 데이터 샘플 5행:")
print(df.head(), "\n")

print("📏 데이터 정보:")
print(df.info(), "\n")

🔍 데이터 샘플 5행:
         date  caregiver_id  caregiver_group  time_punctuality  \
0  2023-01-01             1                1                 4   
1  2023-01-02             1                1                 4   
2  2023-01-03             1                1                 4   
3  2023-01-04             1                1                 5   
4  2023-01-05             1                1                 4   

   preparedness_activity  communication_with_child  safety_management  \
0                      5                         4                  4   
1                      5                         4                  4   
2                      5                         5                  4   
3                      4                         5                  4   
4                      5                         4                  4   

   communication_skill           review_text  days_since_last_rematch  \
0                    3  아이가 잘 따르고 소통도 좋았습니다.                        0   
1    

In [ ]:
# =========================
# 2. 전처리
# =========================

# (1) date를 수치형으로 변환
df["date"] = pd.to_datetime(df["date"])
min_date = df["date"].min()
df["days_from_start"] = (df["date"] - min_date).dt.days

# (2) 사용할 feature 선정
feature_cols = [
    "caregiver_group",
    "time_punctuality",
    "preparedness_activity",
    "communication_with_child",
    "safety_management",
    "communication_skill",
    "days_since_last_rematch",
    "rematched_today",
    "days_from_start",
]

target_col = "rematch_probability"

X = df[feature_cols].copy()
y = df[target_col].copy()

In [ ]:
# =========================
# 3. Train/Test 분할 (시간 순서 보존)
# =========================

df_sorted = df.sort_values("date").reset_index(drop=True)
X_sorted = df_sorted[feature_cols]
y_sorted = df_sorted[target_col]

train_ratio = 0.8
train_size = int(len(df_sorted) * train_ratio)

X_train = X_sorted.iloc[:train_size]
y_train = y_sorted.iloc[:train_size]
X_test = X_sorted.iloc[train_size:]
y_test = y_sorted.iloc[train_size:]

print(f"🧪 Train size: {len(X_train)}, Test size: {len(X_test)}\n")

🧪 Train size: 5848, Test size: 1462



In [ ]:
# =========================
# 4. LightGBM 회귀 모델 정의 & 학습
# =========================

# 하이퍼파라미터는 synthetic 데이터 기준으로 무난한 값으로 설정
# 필요하면 나중에 Optuna나 GridSearch로 튜닝 가능
model = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective="regression"
)

print("🚀 모델 학습 시작...")
model.fit(X_train, y_train)
print("✅ 모델 학습 완료!\n")

#모델 저장
import pickle
with open("model_rematch_probability.pkl", "wb") as f:
    pickle.dump(model, f)

🚀 모델 학습 시작...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000409 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 319
[LightGBM] [Info] Number of data points in the train set: 5848, number of used features: 9
[LightGBM] [Info] Start training from score 97.811366
✅ 모델 학습 완료!

